In [13]:
from selenium import webdriver
from selenium.webdriver.common import actions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
import pandas as pd 
import os 
import sys
from datetime import date, datetime
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import lxml
import time
import requests, pickle

In [14]:
def clicking(path, element='elemento selecionado', refresh=False, by=By.XPATH, limit=3):
    """
    * path: caminho do elemento que será encontrado na página. Pode ser um xpath, um ID, um name e etc;
    * element: nome do elemento que será encontrado na página. Esse parâmetro tem como finalidade unicamente para melhor depuração. Pode ser omitido;
    * refresh: default False - tem como objetivo atualizar a página para tentar clicar no elemento. Utilizado para clicar em arquivos em listas de query;
    * by: default XPATH - método utilizado para localizar o elemento, como XPATH, ID, NAME, CSS_SELECTOR e etc;
    * limit: default 3 - quantidade de vezes que a função irá tentar clicar no elemento. Por padrão, cada tentativa leva 20 segundos. Se o elemento não for encontrado, retornará None;
    """


    if limit == 0:
        return None


    elif refresh:
        driver.refresh()

    try:
        object = driver.find_element(by, path)
        actions = ActionChains(driver)
        actions.move_to_element(object).perform()
        
        found_element = driver.find_element(by, path)
        print(f">>>Sucesso {element}>>>")

    except:
        print(f"<<<Erro ao {element}, tentando novamente>>>")
        found_element = clicking(element=element, path=path, refresh=refresh, by=by, limit=limit - 1)


    return found_element

def create_driver(download_dir=None, headless=False):
    chrome_options = Options()

    if headless:
        chrome_options.headless=True

    if download_dir is not None:
        download_dir=str(download_dir)

    preferences = {"download.default_directory": download_dir,
                   "directory_upgrade": True,
                   "safebrowsing.enabled": True}
                   
    #chrome_options.add_argument("user-data-dir=C:\\Users\\vitor\\AppData\\Local\\Google\\Chrome\\User Data")
    chrome_options.add_argument(f"user-data-dir={os.path.join(os.getcwd(), 'profile', 'wpp')}")
    chrome_options.add_experimental_option("prefs", preferences)
    chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(20)

    return driver

def error(e):
    print("Apresentou erro, gravando o erro")
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    path_erro = r'\\branas06av3-nrd\LAR_BRA_DIGITAL_F\Facens\Logs'
    #path_erro = r"C:\Users\IWX1057086\Documents\Programas\TIM RPA\log"
    today = date.today()
    name = str(fname) + '_' + str(today) + '.txt'
    arquivo = open(path_erro + '\\' + name, 'w')
    arquivo.write(str(fname) + "\n")
    arquivo.write(str(e) + "\n")
    arquivo.write(str(exc_type) + "\n")
    arquivo.write(str(exc_tb.tb_lineno) + "\n")
    arquivo.close()
   

In [15]:
driver = create_driver()

#Login
usr = 'Alineliel'
psw = 'Vitor*06'

driver.get('https://phibo.space')

In [16]:
clicking(element='Passando o usuario', path="/html/body/div[1]/form/div/div/input[1]").send_keys(usr)
clicking(element="Passando a senha", path='/html/body/div[1]/form/div/div/input[2]').send_keys(psw)
clicking(element='Clicando em acessar', path='/html/body/div[1]/form/div/div/button').click()


>>>Sucesso Passando o usuario>>>
>>>Sucesso Passando a senha>>>
>>>Sucesso Clicando em acessar>>>


In [17]:
clicking(element='Clicando em vendas', path='/html/body/div[1]/div[2]/div[1]/div[2]/form/table/tbody/tr/td[1]/div/ul/li[3]/a/span[2]').click()
clicking(element='Clicando em vendas', path='/html/body/div[1]/div[2]/div[1]/div[2]/form/table/tbody/tr/td[1]/div/ul/li[3]/a/span[2]').click()
clicking(element='Clicando em vendas', path='/html/body/div[1]/div[2]/div[1]/div[2]/form/table/tbody/tr/td[1]/div/ul/li[3]/a/span[2]').click()
clicking(element='Clicando em vendas', path='/html/body/div[1]/div[2]/div[1]/div[2]/form/table/tbody/tr/td[1]/div/ul/li[3]/a/span[2]').click()

>>>Sucesso Clicando em vendas>>>
>>>Sucesso Clicando em vendas>>>
>>>Sucesso Clicando em vendas>>>
>>>Sucesso Clicando em vendas>>>


In [18]:
clicking(element='Selecionado Consulta Vendas', path="//span[text()='Consulta Vendas']").click()                                                                    
clicking(element='Selecionando Listar Cartão Fidelidade', path="//span[text()='Listar Cartão Fidelidade']").click()
time.sleep(10)

>>>Sucesso Selecionado Consulta Vendas>>>
>>>Sucesso Selecionando Listar Cartão Fidelidade>>>


In [19]:
clicking(element='Limpando o campo de Quantidade', path='/html/body/div[1]/div[2]/div[3]/form/table/tbody/tr/td[2]/div/span[1]/input[1]').clear()
clicking(element='Limpando o campo de Quantidade', path='/html/body/div[1]/div[2]/div[3]/form/table/tbody/tr/td[2]/div/span[1]/input[1]').send_keys('10,00')

>>>Sucesso Limpando o campo de Quantidade>>>
>>>Sucesso Limpando o campo de Quantidade>>>


In [20]:
clicking(element='Clicando o campo de confirmar', path='/html/body/div[1]/div[2]/div[3]/form/table/tbody/tr/td[3]/a').click()
time.sleep(10)

>>>Sucesso Clicando o campo de confirmar>>>


In [21]:
df_fidelidade = pd.read_html(driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[3]/form/div[2]/div[2]/table").get_attribute('outerHTML'))[0]

### Whatsapp


In [3]:
driver = create_driver(download_dir=None, headless=False)
driver.get('http://web.whatsapp.com/')

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: Unable to receive message from renderer
  (Session info: chrome=104.0.5112.79)
Stacktrace:
Backtrace:
	Ordinal0 [0x008FAF23+2207523]
	Ordinal0 [0x008930E1+1781985]
	Ordinal0 [0x007A3FE8+802792]
	Ordinal0 [0x00796F7C+749436]
	Ordinal0 [0x00795CF4+744692]
	Ordinal0 [0x007962F8+746232]
	Ordinal0 [0x007A0450+787536]
	Ordinal0 [0x0079767D+751229]
	Ordinal0 [0x007982B1+754353]
	Ordinal0 [0x0079788A+751754]
	Ordinal0 [0x00796E0F+749071]
	Ordinal0 [0x00795CF4+744692]
	Ordinal0 [0x007961CD+745933]
	Ordinal0 [0x007A57B4+808884]
	Ordinal0 [0x007FE0AD+1171629]
	Ordinal0 [0x007ED5EC+1103340]
	Ordinal0 [0x007FD9F2+1169906]
	Ordinal0 [0x007ED406+1102854]
	Ordinal0 [0x007C75F0+947696]
	Ordinal0 [0x007C84E6+951526]
	GetHandleVerifier [0x00BA2A12+2733954]
	GetHandleVerifier [0x00B94E28+2677656]
	GetHandleVerifier [0x009846EA+513114]
	GetHandleVerifier [0x00983796+509190]
	Ordinal0 [0x00899CEB+1809643]
	Ordinal

In [40]:
while len(driver.find_element(By.ID, "side").text) < 1:
    time.sleep(1)

In [57]:
numero = '+55 15 99796-0047'
texto = r"FUNCIONAA!!!!!!"

In [58]:
link = f'http://web.whatsapp.com/send?phone={numero}&text={texto}'

In [59]:
driver.get(link)

In [48]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[4]/div/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div/p').send_keys(Keys.ENTER)

In [64]:
driver.find_element(By.CSS_SELECTOR, "#main > footer > div._2BU3P.tm2tP.copyable-area > div > span:nth-child(2) > div > div._3HQNh._1un-p > div._2jitM > div > span > div > div > ul > li:nth-child(4) > button > input[type=file]").send_keys(r'C:\Users\vitor\Documents\Programas\Bot_Controle_Loja\hello_word.png')

In [49]:
driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/div/div[2]/div[2]/div/div').click()